## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
%load_ext line_profiler

In [3]:
# 
import numpy as np

A = np.random.randint(0, 1000, size=1_000_000)
A

array([161, 520, 666, ..., 783, 494, 449])

In [4]:
def f1(A):
    acc, cnt = 0, 0
    for ai in A:
        bi=ai+100
        acc+=bi
        cnt+=1
    return acc/cnt

In [5]:
def f2(A):
    acc = 0
    for ai in A:
        bi=ai+100
        acc+=bi
    return acc/len(A)

In [6]:
def f3(A):
    return sum(A)/len(A) + 100

In [7]:
def f4(A):
    return A.mean()+100

In [8]:
# проверка времени выполнения функции

# %timeit f1(A) 
# %timeit f2(A)
# %timeit f3(A)
# %timeit f4(A)


In [9]:
# %lprun -f f2 f2(A)

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [10]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

In [11]:
%timeit g(df)

661 ms ± 35.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%lprun -f g g(df)

Timer unit: 1e-06 s

Total time: 0.647601 s
File: <ipython-input-10-e94296ffa06f>
Function: g at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def g(df):
    10         1          3.0      3.0      0.0      letters = ['a', 'b', 'c', 'd', 'e']
    11         1          1.0      1.0      0.0      dfs = []
    12         6          5.0      0.8      0.0      for letter in letters:
    13         5     628240.0 125648.0     97.0          q = df[df['key']==letter]
    14         5         11.0      2.2      0.0          dfs.append(q)
    15         1      19341.0  19341.0      3.0      return pd.concat(dfs, axis=0)

In [13]:
def g_op(df):
    return df[df["key"].isin(["a", "b", "c", "d", "e"])]


In [14]:
%timeit g_op(df)

70.2 ms ± 608 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [15]:
import pandas as pd
recipes = pd.read_csv("./data/recipes_sample.csv", sep=",", parse_dates=['submitted'])
reviews = pd.read_csv("./data/reviews_sample.csv", sep=",", parse_dates=['date'])

In [29]:
def f1(reviews):
    """А) С использованием метода `DataFrame.iterrows` исходной таблицы"""
    acc = 0
    for _, row in reviews.iterrows():
        acc += row["rating"]
    return acc/len(reviews)


def f2(reviews):
    """Б) С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год"""
    acc = 0
    for _, row in reviews.iterrows():
        # acc += row.loc["rating"]
        print(row['date'], (pd.to_datetime("01.01.2004", format="%d.%m.%Y") == row['date'])
    # return acc/len(reviews)


def f3(reviews):
    """В) С использованием метода `Series.mean`"""
    return reviews["rating"].mean()


f2(reviews)
# %timeit f3(reviews)
# %timeit f2(reviews)
# %timeit f1(reviews)

# TODO добавить условие 2010 года


2003-05-01 00:00:00 False
2007-09-16 00:00:00 False
2008-01-10 00:00:00 False
2017-12-11 00:00:00 False
2008-03-14 00:00:00 False
2003-01-03 00:00:00 False
2006-12-10 00:00:00 False
2005-12-09 00:00:00 False
2007-07-03 00:00:00 False
2008-12-14 00:00:00 False
2008-09-04 00:00:00 False
2007-11-18 00:00:00 False
2002-04-05 00:00:00 True
2008-08-21 00:00:00 False
2005-10-17 00:00:00 False
2018-04-19 00:00:00 False
2005-07-11 00:00:00 False
2009-11-17 00:00:00 False
2011-04-19 00:00:00 False
2012-07-01 00:00:00 False
2009-01-24 00:00:00 False
2007-03-02 00:00:00 False
2006-06-25 00:00:00 False
2008-02-25 00:00:00 False
2006-12-23 00:00:00 False
2010-12-21 00:00:00 False
2007-02-28 00:00:00 False
2008-05-07 00:00:00 False
2010-06-13 00:00:00 False
2006-02-01 00:00:00 False
2004-10-11 00:00:00 False
2009-09-15 00:00:00 False
2011-11-05 00:00:00 False
2003-05-14 00:00:00 False
2008-10-27 00:00:00 False
2008-05-06 00:00:00 False
2008-12-24 00:00:00 False
2008-02-19 00:00:00 False
2009-11-03 00

KeyboardInterrupt: 

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [ ]:
%lprun -f f1 f1(reviews)

Timer unit: 1e-06 s

Total time: 30.4085 s
File: <ipython-input-33-142e62787846>
Function: f1 at line 18

Line #      Hits         Time  Per Hit   % Time  Line Contents
    18                                           def f1(reviews):
    19                                               """Метод iterrows без использования срезов таблицы"""
    20         1          2.0      2.0      0.0      acc = 0
    21    126697   22710992.0    179.3     74.7      for _, row in reviews.iterrows():
    22    126696    7697526.0     60.8     25.3          acc+=row.loc["rating"]
    23         1          9.0      9.0      0.0      return acc/len(reviews)

In [ ]:

%lprun -f f2 f2(reviews)

Timer unit: 1e-06 s

Total time: 27.0388 s
File: <ipython-input-33-142e62787846>
Function: f2 at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def f2(reviews):
     7                                               """Метод iterrows с использованием срезов таблицы"""
     8         1          1.0      1.0      0.0      acc = 0
     9    126697   22689705.0    179.1     83.9      for _, row in reviews.iterrows():
    10    126696    4349066.0     34.3     16.1          acc+=row["rating"]
    11         1          9.0      9.0      0.0      return acc/len(reviews)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

%timeit get_word_reviews_count(reviews)

KeyboardInterrupt: 

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.
